In [40]:
import pandas as pd
import re
#import unidecode
import numpy as np
import matplotlib.pyplot as plt
import datetime
from google.colab import drive
from tqdm.notebook import tqdm
tqdm.pandas()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
#Seasons in data
seasons = [2021,2020,2019,2018,2017,2016,2015]
#How many time steps to include in input
day_look_back = 30

In [42]:
injury_df = pd.read_csv('/content/drive/MyDrive/data_490/injury_data/scraped_injury.csv')

In [43]:
injury_df[injury_df.player_name == 'Joe Biagini']

,player_name,date1,date2,date3,date4,date5,date6,date7,date8,date9,...,injury_type4,injury_type5,injury_type6,injury_type7,injury_type8,injury_type9,injury_type10,injury_type11,injury_type12,injury_type13
635,Joe Biagini,2018-09-04,2020-07-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
#Used to itterate through injury columns
num_injury_columns = len(injury_df.columns) // 2

In [45]:
#unique injury holds all the unique injury types for each injury occurence
unique_injury = np.empty(0)
for i in range(1,num_injury_columns+1):
  unique_injury = np.append(unique_injury, injury_df[f'injury_type{i}'].unique())

In [46]:
#Drops nan values
unique_injury = unique_injury[~pd.isna(unique_injury)]

In [47]:
#All unique injury types
unique_injury = np.unique(unique_injury)

In [48]:
#Data frame with columns for each type of injury and player name
injury_history = pd.DataFrame(columns=unique_injury)
injury_history['player_name'] = []

In [49]:
#row is passed from injury_df
#Each column is a type of injury and the date when that injury occured for each player
def get_injury_history(row):
  for i in range(1,num_injury_columns+1):
    injury = row[f'injury_type{i}']
    injury_date = row[f'date{i}']
    if (injury != injury) or (injury_date != injury_date):
      continue
    else:
      injury_history.loc[get_injury_history.i, injury] = injury_date
      injury_history.loc[get_injury_history.i, 'player_name'] = row.player_name
  get_injury_history.i += 1
  return

In [50]:
get_injury_history.i = 0
injury_df.apply(get_injury_history,axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1297    None
1298    None
1299    None
1300    None
1301    None
Length: 1302, dtype: object

In [51]:
for column in injury_history.iloc[:,:-1]:
  injury_history[column] = pd.to_datetime(injury_history[column])

In [52]:
injury_history

,abdominal,achilles,ankle,arm,back,biceps,calf,chest,collarbone,concussion,...,ribs,shin,shoulder,thigh,throat,thumb,toe,undisclosed,wrist,player_name
0,NaT,NaT,NaT,NaT,NaT,NaT,2013-06-13,NaT,NaT,NaT,...,NaT,NaT,2007-07-01,NaT,NaT,NaT,NaT,2012-04-04,NaT,A.J. Burnett
1,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,2019-08-10,NaT,NaT,NaT,NaT,NaT,NaT,A.J. Cole
2,NaT,NaT,2017-04-21,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,2016-05-08,NaT,NaT,NaT,NaT,2017-05-27,NaT,A.J. Griffin
3,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,2019-09-12,NaT,NaT,NaT,NaT,NaT,NaT,A.J. Minter
4,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,2016-06-16,NaT,NaT,NaT,NaT,NaT,NaT,A.J. Morris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297,NaT,2018-02-16,NaT,2017-05-06,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,2012-03-26,NaT,NaT,NaT,NaT,NaT,NaT,Zack Britton
1298,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Zack Godley
1299,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2013-04-12,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2011-03-26,NaT,Zack Greinke
1300,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,Zack Littell


In [53]:
df = pd.read_csv('/content/drive/MyDrive/data_490/processed_data/pandas_30_day')
print(len(df))
df.game_date = pd.to_datetime(df.game_date)
df = df[['player_name', 'game_date']]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (89,95,96,97,99,105,106,107) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


132274


In [54]:
#Creates a data frame where each row is a player appereance with player name game date and players injury history
injury_history = df.merge(injury_history, how='left', on='player_name')

In [55]:
#Each injury column holds the amount of time between game_date and injury_date
for column in injury_history.iloc[:,2:].columns:
  injury_history[column] = injury_history.game_date - injury_history[column]

In [56]:
injury_history

,player_name,game_date,abdominal,achilles,ankle,arm,back,biceps,calf,chest,...,quad,ribs,shin,shoulder,thigh,throat,thumb,toe,undisclosed,wrist
0,A.J. Achter,2015-08-07,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,A.J. Achter,2015-08-08,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,A.J. Achter,2015-08-12,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,A.J. Achter,2015-08-14,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,A.J. Achter,2015-08-18,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132269,Zach Duke,2016-05-09,NaT,NaT,3908 days,NaT,NaT,NaT,-1107 days,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
132270,Zack Godley,2016-09-17,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
132271,Erick Aybar,2017-04-18,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,1726 days,NaT,NaT
132272,Steven Wright,2018-09-16,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,763 days,NaT,NaT,NaT,-301 days,NaT,NaT


In [57]:
#Converts data from date time to number of days as intergers
for column in injury_history.iloc[:,2:].columns:
  injury_history[column] = injury_history[column] / datetime.timedelta(days=1)

In [58]:
#Replaces injury dates that happened after or on game date with nan
for column in injury_history.iloc[:,2:].columns:
  injury_history[column] = injury_history[column].where(injury_history[column] >= 0, np.nan)

In [64]:
injury_history

,player_name,game_date,abdominal,achilles,ankle,arm,back,biceps,calf,chest,...,quad,ribs,shin,shoulder,thigh,throat,thumb,toe,undisclosed,wrist
0,A.J. Achter,2015-08-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A.J. Achter,2015-08-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A.J. Achter,2015-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A.J. Achter,2015-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A.J. Achter,2015-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132269,Zach Duke,2016-05-09,NaN,NaN,3908.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132270,Zack Godley,2016-09-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132271,Erick Aybar,2017-04-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1726.0,NaN,NaN
132272,Steven Wright,2018-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,763.0,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
injury_history.to_csv('/content/drive/MyDrive/data_490/injury_data/injury_history', index=False)